In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('data_healthcare/dataset.csv')
desc_sym = pd.read_csv('data_healthcare/symptom_Description.csv')
prec_sym = pd.read_csv('data_healthcare/symptom_precaution.csv')
sym_wight = pd.read_csv('data_healthcare/Symptom-severity.csv')

In [3]:
#mengilangkan white space
dataset = dataset.apply(lambda x: x.str.strip())
dataset = dataset.replace('_ ','_', regex=True).replace(' _','_', regex=True).replace(' ','_', regex=True) #menghilangkan space pada symptom
desc_sym = desc_sym.apply(lambda x: x.str.strip())
prec_sym = prec_sym.apply(lambda x: x.str.strip())

## EDA

In [4]:
dataset.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal_infection,itching,skin_rash,nodal_skin_eruptions,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal_infection,skin_rash,nodal_skin_eruptions,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal_infection,itching,nodal_skin_eruptions,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal_infection,itching,skin_rash,dischromic_patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal_infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(desc_sym.isna().sum())
desc_sym.head()

Disease        0
Description    0
dtype: int64


,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [6]:
print(prec_sym.isna().sum())
prec_sym.head()

Disease         0
Precaution_1    0
Precaution_2    0
Precaution_3    1
Precaution_4    1
dtype: int64


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


Jumlah Penyakit

In [7]:
len(dataset.Disease.unique()), len(desc_sym.Disease.unique()), len(prec_sym.Disease.unique())

(41, 41, 41)

Jumlah kasus setiap Penyakit

In [8]:
dataset.Disease.value_counts()

Fungal_infection                           120
Hepatitis_C                                120
Hepatitis_E                                120
Alcoholic_hepatitis                        120
Tuberculosis                               120
Common_Cold                                120
Pneumonia                                  120
Dimorphic_hemmorhoids(piles)               120
Heart_attack                               120
Varicose_veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo)_Paroymsal__Positional_Vertigo    120
Acne                                       120
Urinary_tract_infection                    120
Psoriasis                                  120
Hepatitis_D                                120
Hepatitis_B                                120
Allergy      

Jumlah gejala pada dataset.csv

In [9]:
import itertools
all_symptom = dataset.drop(columns=['Disease']).values.tolist()
all_symptom = list(itertools.chain.from_iterable(all_symptom))
all_symptom = list(set(all_symptom))
all_symptom = [x for x in all_symptom if str(x) != 'nan']
print('Total Gejala di dataset:',len(all_symptom))

Total Gejala di dataset: 131


Gejala setiap penyakit

In [10]:
# Banyak gejala disetiap kasus penyakit
df_min_max = pd.DataFrame()
for d in dataset.Disease.unique():
  min_sym = (len(dataset.columns)-1)-int(dataset.loc[dataset.Disease == d].isna().sum(axis=1).max())
  max_sym = (len(dataset.columns)-1)-int(dataset.loc[dataset.Disease == d].isna().sum(axis=1).min())
  df_symps = {'Disease': d, 'jmlh_symptom_min': [min_sym], 'jumlh_symptom_max': [max_sym]}
  df_symps = pd.DataFrame(df_symps)
  df_min_max = pd.concat([df_min_max, df_symps])

# Gejala-gejala disetiap penyakit
df_symp_dis = pd.DataFrame()
for d in dataset.Disease.unique():
  symp = dataset.loc[dataset.Disease == d]
  symp = symp.drop(columns=['Disease']).values.tolist()
  symp = list(itertools.chain.from_iterable(symp))
  symp = list(set(symp))
  symp = [x for x in symp if str(x) != 'nan']
  df_symp = {'Disease':d, 'total_symptom':len(symp), 'symptoms':[symp]}
  df_symp = pd.DataFrame(df_symp)
  df_symp_dis = pd.concat([df_symp_dis, df_symp])


# Eksplorasi Gejala penyakit
pd.merge(df_min_max, df_symp_dis, on=["Disease"]).sort_values(by='total_symptom', ascending=False).reset_index(drop=True)


,Disease,jmlh_symptom_min,jumlh_symptom_max,total_symptom,symptoms
0,Common_Cold,16,17,17,"[throat_irritation, muscle_pain, malaise, swel..."
1,Tuberculosis,15,16,16,"[weight_loss, loss_of_appetite, vomiting, mala..."
2,Dengue,13,14,14,"[nausea, loss_of_appetite, vomiting, muscle_pa..."
3,Hepatitis_E,12,13,13,"[nausea, loss_of_appetite, vomiting, abdominal..."
4,Hypothyroidism,12,13,13,"[cold_hands_and_feets, depression, mood_swings..."
5,Hepatitis_B,11,12,12,"[itching, loss_of_appetite, receiving_unsteril..."
6,Hypoglycemia,11,12,12,"[nausea, blurred_and_distorted_vision, vomitin..."
7,hepatitis_A,10,11,11,"[nausea, loss_of_appetite, vomiting, muscle_pa..."
8,Typhoid,10,11,11,"[nausea, vomiting, abdominal_pain, diarrhoea, ..."
9,Chicken_pox,10,11,11,"[itching, loss_of_appetite, malaise, swelled_l..."


Jumlah gejala pada Symptom-severity.csv

In [11]:
print( 'Jumlah data :', len(sym_wight.Symptom.values.tolist()))
print( 'Jumlah gejala :', len(sym_wight.Symptom.unique().tolist()))

Jumlah data : 133
Jumlah gejala : 132


Terdapat satu gejala yang mempunyai dua weight

In [12]:
#gejala duplicate
import collections
print([sym for sym, count in collections.Counter(sym_wight.Symptom.values.tolist()).items() if count > 1])

['fluid_overload']


In [13]:
sym_wight.loc[sym_wight.Symptom == 'fluid_overload']

,Symptom,weight
45,fluid_overload,6
117,fluid_overload,4


Melakukan cek gejala yang tidak memiliki weight

In [14]:
sym_no_wight=[]
for sym in all_symptom:
  if sym not in sym_wight.Symptom.unique().tolist():
    sym_no_wight.append(sym)
print('Gejala tidak mempunyai weight:',sym_no_wight)

Gejala tidak mempunyai weight: ['foul_smell_of_urine']


Rangkuman EDA

1. Terdapat 41 jenis Penyakit pada Dataset 
2. Jumlah dataset seimbang dengan data setiap penyakit sebanyak 120
3. Setiap penyakit memiliki gejala dengan gejala paling sedikit sebanyak 3 dan maksimum 17 gejala
4. Terdapat 131 Gejala pada dataset
5. Pada Symptom-severity.csv terdapat gejala yang mempunyai 2 weight : ['fluid_overload']
6. Gejala ['foul_smell_of_urine'] tidak mempunyai weight

##  feature extraction